In [73]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np
from graphgym.custom_graphgym.loader.protein_batch import ProteinBatchDataset
import matplotlib.pyplot as plt
import os

In [74]:
# General imports
import os
import json
import collections

# Data science imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import scipy.sparse as sp

# Import Weights & Biases for Experiment Tracking
import wandb

# Graph imports
import torch
from torch import Tensor
import torch.nn.functional as F

import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.utils import to_networkx

import networkx as nx
from networkx.algorithms import community

from tqdm.auto import trange

In [75]:
os.chdir('/Users/cgu3/Documents/Grape-Pi')

In [76]:
protein_dataset = ProteinBatchDataset("data/protein_graphs", rebuild=True)
# protein_dataset = ProteinBatchDataset("data/TCE_graphs", rebuild=True)
# protein_dataset = ProteinBatchDataset("data/localization", rebuild=True)

Rebuilding...


KeyboardInterrupt: 

In [ ]:
len(protein_dataset.y)

In [ ]:
import random

random.seed(1234)

ids = list(range(0, len(protein_dataset.y)))
train_ids = random.sample(ids, int(0.80 * len(ids)))
test_ids = list(set(ids) - set(train_ids))

In [ ]:
from torch_geometric.loader import DataLoader
train_loader = DataLoader(protein_dataset[train_ids], batch_size=10, shuffle=True)
test_loader = DataLoader(protein_dataset[test_ids], batch_size=10, shuffle=False)

for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

In [ ]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(1, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        # self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin1 = Linear(hidden_channels, 128)
        self.lin2 = Linear(128, 64)
        self.lin3 = Linear(64, 3) # (previous_dim, num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        # x = self.conv3(x, edge_index)
        # x = x.relu()
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        # x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin1(x)
        x = x.relu()
        x = self.lin2(x)
        x = x.relu()
        x = self.lin3(x)
        
        return x

model = GCN(hidden_channels=64)
print(model)

from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))

model = GCN(hidden_channels=256)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.x, data.edge_index, data.batch)  # Perform a single forward pass.
         loss = criterion(out, data.y)  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.
        
def test(loader):
     model.eval()
     correct = 0
     loss_ = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data.x, data.edge_index, data.batch)  
         loss = criterion(out, data.y)
         loss_ += loss.item()
         pred = out.argmax(dim=1)  # Use the class with highest probability.
         
         correct += int((pred == data.y).sum())  # Check against ground-truth labels.
     return correct / len(loader.dataset), loss_ / len(loader.dataset)  # Derive ratio of correct predictions.

In [ ]:
for epoch in trange(1, 101):
    train()
    train_acc, train_loss = test(train_loader)
    test_acc, test_loss = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train loss: {train_loss:.4f}, Test loss: {test_loss:.4f}')
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

In [91]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load data
data = pd.read_csv('/Users/cgu3/Documents/Grape-Pi/data/protein_graphs/raw/proteins.csv', index_col=0)  # Replace with your file path
with open('/Users/cgu3/Documents/Grape-Pi/data/protein_graphs/raw/graph_labels.txt') as f:
    graph_labels = f.read().splitlines()
graph_labels = [int(label) for label in graph_labels]

# Assuming the last column is the target variable
X = np.transpose(data.values)
y = graph_labels

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [92]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Convert arrays to PyTorch tensors
train_data = TensorDataset(torch.FloatTensor(X_train), torch.LongTensor(y_train))
test_data = TensorDataset(torch.FloatTensor(X_test), torch.LongTensor(y_test))

# Create DataLoaders
batch_size = 32  # You can modify this
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)


In [103]:
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create the model instance
input_size = X_train.shape[1]
hidden_size = 50  # Example size, adjust as needed
output_size = 3  # Adjust based on your task
model = MLP(input_size, hidden_size, output_size)


In [104]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()  # For regression tasks, use MSE. For classification, use CrossEntropyLoss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 100  # Example, adjust as needed
for epoch in range(epochs):
    for inputs, labels in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # # Print loss for every epoch
        # print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')
        # # print accuracy for every epoch
        # _, predicted = torch.max(outputs.data, 1)
        # print('Accuracy: %.2f %%' % (100 * torch.sum(labels==predicted) / len(labels)))

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            outputs = model(inputs)
            # ... add evaluation metrics relevant to your task
            # evaluate accuracy
            predicted = torch.argmax(outputs, axis=1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        print('Test accuracy: {} %'.format(100 * correct / total))


MLP(
  (fc1): Linear(in_features=1119, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=3, bias=True)
)

Test accuracy: 69.23076923076923 %


MLP(
  (fc1): Linear(in_features=1119, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=3, bias=True)
)

Test accuracy: 69.23076923076923 %


MLP(
  (fc1): Linear(in_features=1119, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=3, bias=True)
)

...

Test accuracy: 84.61538461538461 %


MLP(
  (fc1): Linear(in_features=1119, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=3, bias=True)
)

Test accuracy: 84.61538461538461 %


MLP(
  (fc1): Linear(in_features=1119, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=3, bias=True)
)

Test accuracy: 84.61538461538461 %


MLP(
  (fc1): Linear(in_features=1119, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=3, bias=True)
)

Test accuracy: 34.61538461538461 %
